In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile /content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/phomt_vocab.py
import json
import os
from collections import Counter
from itertools import chain

class Vocab:
    def __init__(self, path, src_language, tgt_language, min_freq=5):
        self.src_language = src_language
        self.tgt_language = tgt_language
        self.min_freq = min_freq

        self.pad_token = "<pad>"
        self.unk_token = "<unk>"
        self.bos_token = "<bos>"
        self.eos_token = "<eos>"

        self.pad_idx = 0
        self.unk_idx = 1
        self.bos_idx = 2
        self.eos_idx = 3

        self.src_s2i = {self.pad_token: self.pad_idx, self.unk_token: self.unk_idx}
        self.src_i2s = {self.pad_idx: self.pad_token, self.unk_idx: self.unk_token}
        self.tgt_s2i = {self.pad_token: self.pad_idx, self.unk_token: self.unk_idx, self.bos_token: self.bos_idx, self.eos_token: self.eos_idx}
        self.tgt_i2s = {self.pad_idx: self.pad_token, self.unk_idx: self.unk_token, self.bos_idx: self.bos_token, self.eos_idx: self.eos_token}

        self.build_vocab(path)

    def load_data(self, path):
        files = ["small-train.json", "small-dev.json", "small-test.json"]
        data = []
        for file in files:
            full_path = os.path.join(path, file)
            if os.path.exists(full_path):
                with open(full_path, "r", encoding="utf-8") as f:
                    data.extend(json.load(f))
        return data

    def build_vocab(self, path):
        data = self.load_data(path)

        src_tokens = [item[self.src_language].split() for item in data]
        tgt_tokens = [item[self.tgt_language].split() for item in data]

        src_counter = Counter(chain.from_iterable(src_tokens))
        tgt_counter = Counter(chain.from_iterable(tgt_tokens))

        # Xây dựng từ điển Source (tiếng Anh)
        for token, count in src_counter.items():
            if count >= self.min_freq and token not in self.src_s2i:
                idx = len(self.src_s2i)
                self.src_s2i[token] = idx
                self.src_i2s[idx] = token

        # Xây dựng từ điển Target (tiếng Việt)
        for token, count in tgt_counter.items():
            if count >= self.min_freq and token not in self.tgt_s2i:
                idx = len(self.tgt_s2i)
                self.tgt_s2i[token] = idx
                self.tgt_i2s[idx] = token

        self.src_vocab_size = len(self.src_s2i)
        self.tgt_vocab_size = len(self.tgt_s2i)

    def encode(self, text, is_target=False):
        tokens = text.split()
        s2i = self.tgt_s2i if is_target else self.src_s2i
        unk_idx = self.unk_idx

        indices = [s2i.get(token, unk_idx) for token in tokens]

        if is_target:
            indices = [self.bos_idx] + indices + [self.eos_idx]

        return indices

    def decode(self, indices, is_target=False):
        i2s = self.tgt_i2s if is_target else self.src_i2s
        tokens = [i2s.get(idx, self.unk_token) for idx in indices]
        return " ".join(tokens)

Overwriting /content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/phomt_vocab.py


In [ ]:
%%writefile /content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/phomt_dataset.py
import json
import torch
from torch.utils.data import Dataset, DataLoader
import os
import sys

# Đảm bảo có thể import Vocab nếu file này được chạy độc lập
if __name__ != "__main__":
    from phomt_vocab import Vocab
else:
    # Thêm đường dẫn dự án để import được Vocab khi chạy file này độc lập
    sys.path.append(os.path.dirname(os.path.abspath(__file__)))
    try:
        from phomt_vocab import Vocab
    except ImportError:
        print("Không tìm thấy phomt_vocab.py. Đảm bảo nó cùng thư mục.")
        exit()


class phoMTDataset(Dataset):
    def __init__(self, data_path, vocab):
        self.vocab = vocab
        with open(data_path, "r", encoding="utf-8") as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        src_text = item[self.vocab.src_language]
        tgt_text = item[self.vocab.tgt_language]

        # Chuyển đổi thành indices
        src_indices = self.vocab.encode(src_text, is_target=False)
        tgt_indices = self.vocab.encode(tgt_text, is_target=True)

        return {
            'src': torch.tensor(src_indices, dtype=torch.long),
            'tgt': torch.tensor(tgt_indices, dtype=torch.long)
        }

def collate_fn(batch):
    # Tìm chiều dài lớn nhất của câu trong batch
    src_lens = [len(item['src']) for item in batch]
    tgt_lens = [len(item['tgt']) for item in batch]
    max_src_len = max(src_lens)
    max_tgt_len = max(tgt_lens)

    # Lấy pad index (giả định pad_idx là 0)
    pad_idx = 0

    # Padding
    padded_src = torch.full((len(batch), max_src_len), pad_idx, dtype=torch.long)
    padded_tgt = torch.full((len(batch), max_tgt_len), pad_idx, dtype=torch.long)

    for i, item in enumerate(batch):
        padded_src[i, :src_lens[i]] = item['src']
        padded_tgt[i, :tgt_lens[i]] = item['tgt']

    return {
        'src': padded_src,
        'tgt': padded_tgt
    }

Overwriting /content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/phomt_dataset.py


In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 24.1 MB/s eta 0:00:00


In [ ]:
%%writefile /content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/seq2seq_luong_lstm.py
import torch
from torch import nn
import torch.nn.functional as F

# --- 1. ENCODER ---
class EncoderLSTM(nn.Module):
    def __init__(self, vocab_size, d_model, n_encoder, dropout, pad_idx):
        super().__init__()
        self.d_model = d_model
        self.n_encoder = n_encoder
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)

        # LSTM Unidirectional
        self.lstm = nn.LSTM(
            input_size=d_model,
            hidden_size=d_model,
            num_layers=n_encoder,
            dropout=dropout if n_encoder > 1 else 0,
            batch_first=True,
            bidirectional=False
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src: (batch_size, src_len)
        embedded = self.dropout(self.embedding(src))
        # embedded: (batch_size, src_len, d_model)

        # output: (batch_size, src_len, d_model) -> output này chính là H_s dùng trong Attention
        # hidden: (h_n, c_n): (n_encoder, batch_size, d_model)
        output, hidden = self.lstm(embedded)

        return output, hidden

# --- 2. LUONG ATTENTION MODULE ---
class LuongAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        # Luong Attention sử dụng cơ chế "Dot Product" nếu d_model bằng nhau
        self.d_model = d_model

    def forward(self, decoder_output, encoder_outputs):
        # decoder_output (Q): H_t (trạng thái ẩn hiện tại của decoder) - (batch_size, 1, d_model)
        # encoder_outputs (K): H_s (các trạng thái ẩn của encoder) - (batch_size, src_len, d_model)

        # Tính điểm năng lượng (energy scores): score(H_t, H_s)
        # Luong 'dot' score: H_t * H_s^T
        # scores: (batch_size, 1, src_len)
        scores = torch.bmm(decoder_output, encoder_outputs.transpose(1, 2))

        # Tính trọng số Attention: attention_weights
        # attention_weights: (batch_size, 1, src_len) -> tổng theo chiều src_len = 1
        attention_weights = F.softmax(scores, dim=-1)

        # Tính Context Vector (C_t): Context = attention_weights * H_s
        # context_vector: (batch_size, 1, d_model)
        context_vector = torch.bmm(attention_weights, encoder_outputs)

        return context_vector, attention_weights

# --- 3. DECODER VỚI ATTENTION ---
class AttentionDecoderLSTM(nn.Module):
    def __init__(self, vocab_size, d_model, n_decoder, dropout, pad_idx):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.n_decoder = n_decoder

        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)

        # LSTM nhận đầu vào là (embedded + context) nhưng ở đây ta dùng embedded (d_model)
        # và kết hợp context sau bước LSTM (Luông)
        self.lstm = nn.LSTM(
            input_size=d_model,
            hidden_size=d_model,
            num_layers=n_decoder,
            dropout=dropout if n_decoder > 1 else 0,
            batch_first=True,
            bidirectional=False
        )

        self.attention = LuongAttention(d_model)

        # Layer kết hợp output của LSTM và Context vector
        # Input size: 2 * d_model (h_t + C_t)
        self.concat_layer = nn.Linear(2 * d_model, d_model)

        # Layer dự đoán cuối cùng
        self.output_layer = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward_step(self, input_token, hidden, encoder_outputs):
        # input_token: (batch_size, 1)
        # hidden: (h_n, c_n) từ bước trước, mỗi tensor shape: (n_decoder, batch_size, d_model)
        # encoder_outputs: (batch_size, src_len, d_model)

        embedded = self.dropout(self.embedding(input_token))
        # embedded: (batch_size, 1, d_model)

        # 1. LSTM Step
        # output: (batch_size, 1, d_model) -> H_t
        # hidden: (n_decoder, batch_size, d_model) -> H_n, C_n cho bước tiếp theo
        output, hidden = self.lstm(embedded, hidden)

        # 2. Attention
        # context: (batch_size, 1, d_model)
        # attn_weights: (batch_size, 1, src_len)
        context, attn_weights = self.attention(output, encoder_outputs)

        # 3. Concatenation (H_t và C_t) và Projection
        # Concatenated: (batch_size, 1, 2*d_model)
        concat_input = torch.cat((output, context), dim=-1)

        # Concat -> Tanh -> Projected: (batch_size, 1, d_model)
        output_att = torch.tanh(self.concat_layer(concat_input))

        # 4. Final Prediction Layer
        # prediction: (batch_size, 1, vocab_size)
        prediction = self.output_layer(output_att)

        return prediction, hidden, attn_weights

    def forward(self, tgt, initial_hidden, encoder_outputs):
        # tgt: (batch_size, tgt_len) - Decoder input (<bos> y1 y2 ...)
        batch_size, tgt_len = tgt.shape

        # tensor lưu trữ logits dự đoán
        all_predictions = torch.zeros(batch_size, tgt_len, self.vocab_size, device=tgt.device)

        # Hidden state ban đầu lấy từ Encoder
        hidden = initial_hidden

        # Chạy Decoder theo từng bước thời gian (teacher forcing)
        for t in range(tgt_len):
            # input_token: (batch_size, 1)
            input_token = tgt[:, t].unsqueeze(1)

            # prediction: (batch_size, 1, vocab_size)
            prediction, hidden, _ = self.forward_step(input_token, hidden, encoder_outputs)

            # Lưu trữ prediction
            all_predictions[:, t] = prediction.squeeze(1)

        return all_predictions, hidden


class Seq2SeqAttentionLSTM(nn.Module):
    def __init__(self, d_model, n_encoder, n_decoder, dropout, vocab):
        super().__init__()
        self.vocab = vocab
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Encoder (không đổi)
        self.encoder = EncoderLSTM(
            vocab_size=vocab.src_vocab_size,
            d_model=d_model,
            n_encoder=n_encoder,
            dropout=dropout,
            pad_idx=vocab.pad_idx
        )

        # Decoder (Attention)
        self.decoder = AttentionDecoderLSTM(
            vocab_size=vocab.tgt_vocab_size,
            d_model=d_model,
            n_decoder=n_decoder,
            dropout=dropout,
            pad_idx=vocab.pad_idx
        )

    def forward(self, src, tgt):
        # src: (batch_size, src_len)
        # tgt: (batch_size, tgt_len)

        # encoder_outputs: (batch_size, src_len, d_model) - Dùng cho Attention
        # encoder_hidden: (h_n, c_n): (n_layers, batch_size, d_model) - Dùng làm initial hidden
        encoder_outputs, encoder_hidden = self.encoder(src)

        # output: (batch_size, tgt_len, vocab_size)
        output, _ = self.decoder(tgt, encoder_hidden, encoder_outputs)

        return output

    @torch.no_grad()
    def predict(self, src, max_len=50):
        self.eval()
        batch_size = src.shape[0]

        # 1. Encoding
        encoder_outputs, hidden = self.encoder(src)

        # 2. Decoding - Bắt đầu với <bos> token
        input_token = torch.full((batch_size, 1), self.vocab.bos_idx, dtype=torch.long, device=self.device)

        output_tokens = torch.zeros((batch_size, max_len), dtype=torch.long, device=self.device)
        output_tokens[:, 0] = self.vocab.bos_idx

        for t in range(1, max_len):
            # prediction: (batch_size, 1, vocab_size)
            # hidden: state mới
            prediction, hidden, _ = self.decoder.forward_step(input_token, hidden, encoder_outputs)

            # Lấy token có xác suất cao nhất: next_token (batch_size, 1)
            next_token = prediction.argmax(dim=-1)

            # Lưu token vào tensor kết quả
            output_tokens[:, t] = next_token.squeeze(-1)

            # Dừng nếu tất cả các câu trong batch đã sinh ra <eos>
            if ((output_tokens[:, t] == self.vocab.eos_idx) | (output_tokens[:, t] == self.vocab.pad_idx)).all():
                break

            # Cập nhật input cho bước thời gian tiếp theo
            input_token = next_token # (batch_size, 1)

        return output_tokens

Writing /content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/seq2seq_luong_lstm.py


In [ ]:
%%writefile /content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/train_luong_lstm.py
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Subset
import numpy as np
import os
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt

# Import các thành phần cơ bản
from phomt_dataset import collate_fn, phoMTDataset
from phomt_vocab import Vocab
# ĐỔI: Import mô hình Attention mới
from seq2seq_attention_lstm import Seq2SeqAttentionLSTM

# Import Metrics
try:
    from torchmetrics.text.rouge import ROUGEScore
except ImportError:
    print("Vui lòng cài đặt torchmetrics: pip install torchmetrics")
    exit()

# --- Config ---
device = "cuda" if torch.cuda.is_available() else "cpu"
# CẬP NHẬT ĐƯỜNG DẪN CHECKPOINT MỚI
CHECKPOINT_DIR = "/content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/MODEL-LUONG/"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
DATASET_ROOT = "/content/drive/MyDrive/DATASET/small-PhoMT/"


# --- Logging Setup (Sử dụng format chỉ Message cho Console) ---
logger = logging.getLogger()
logger.setLevel(logging.INFO)
if logger.hasHandlers():
    logger.handlers.clear()

file_formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
file_handler = logging.FileHandler(os.path.join(CHECKPOINT_DIR, "training.log"), mode='a')
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

console_formatter = logging.Formatter("%(message)s")
console_handler = logging.StreamHandler()
console_handler.setFormatter(console_formatter)
logger.addHandler(console_handler)
# --- Kết thúc Logging Setup ---

def indices_to_text(indices, vocab, is_target=True):
    tokens = []
    i2s = vocab.tgt_i2s if is_target else vocab.src_i2s

    for idx in indices:
        if isinstance(idx, torch.Tensor):
            idx = idx.item()

        if is_target and idx == vocab.eos_idx:
            break

        if idx != vocab.pad_idx:
            if is_target and idx == vocab.bos_idx:
                continue

            token = i2s.get(idx, vocab.unk_token)
            tokens.append(token)

        if not is_target and idx == vocab.eos_idx:
            break

    return " ".join(tokens)

# Hàm train/evaluate không thay đổi logic, chỉ đổi mô hình
def train(model: nn.Module, dataloader: DataLoader, epoch: int, loss_fn, optimizer):
    model.train()
    running_loss = []

    with tqdm(dataloader, desc=f"Epoch {epoch} - Training") as pbar:
        for item in pbar:
            src = item['src'].to(device)
            tgt = item['tgt'].to(device)

            optimizer.zero_grad()

            decoder_input = tgt[:, :-1]
            targets = tgt[:, 1:]

            logits = model(src, decoder_input)

            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), targets.reshape(-1))

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            running_loss.append(loss.item())
            pbar.set_postfix({"loss": np.mean(running_loss)})

            if device == "cuda":
                torch.cuda.empty_cache()

    avg_loss = np.mean(running_loss)
    logging.info(f"--- Epoch {epoch} TRAIN finished --- Loss: {avg_loss:.4f}")
    return avg_loss

def evaluate(model: nn.Module, dataloader: DataLoader, epoch: int, loss_fn, vocab):
    model.eval()
    running_loss = []
    rouge_metric = ROUGEScore(rouge_keys=("rougeL",)).to(device)
    all_preds_text = []
    all_targets_text = []

    example_printed = False

    with torch.no_grad():
        for item in tqdm(dataloader, desc=f"Epoch {epoch} - Evaluating"):
            src = item['src'].to(device)
            tgt = item['tgt'].to(device)

            # 1. Validation Loss (Sử dụng Teacher Forcing)
            decoder_input = tgt[:, :-1]
            targets = tgt[:, 1:]
            logits = model(src, decoder_input)
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), targets.reshape(-1))
            running_loss.append(loss.item())

            # 2. ROUGE-L Prediction (Sử dụng Inference)
            generated_tokens = model.predict(src, max_len=tgt.shape[1] + 10)

            for i in range(len(tgt)):
                pred_seq = generated_tokens[i].tolist()
                pred_text = indices_to_text(pred_seq, vocab, is_target=True)

                tgt_seq = tgt[i].tolist()
                tgt_text = indices_to_text(tgt_seq, vocab, is_target=True)

                # LOGIC IN VÍ DỤ
                if not example_printed:
                    src_seq = src[i].tolist()
                    src_text = indices_to_text(src_seq, vocab, is_target=False)

                    logging.info(f"\n======== Example Translation (Epoch {epoch}) ========")
                    logging.info(f"-> Source (EN):     {src_text}")
                    logging.info(f"-> Reference (VN):  {tgt_text}")
                    logging.info(f"-> Prediction (VN): {pred_text}")
                    logging.info("==================================================")
                    example_printed = True

                all_preds_text.append(pred_text)
                all_targets_text.append(tgt_text)

            if device == "cuda":
                torch.cuda.empty_cache()

    # Tính ROUGE trên tập Validation
    if len(all_preds_text) > 0:
        rouge_scores = rouge_metric(all_preds_text, all_targets_text)
        rouge_l = rouge_scores['rougeL_fmeasure'].item()
    else:
        rouge_l = 0.0

    avg_loss = np.mean(running_loss)
    logging.info(f"--- Epoch {epoch} EVAL finished --- Val Loss: {avg_loss:.4f} | ROUGE-L: {rouge_l:.4f}")

    return avg_loss, rouge_l

def visualize_metrics(train_losses, val_losses, rouge_scores):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(15, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss', marker='o')
    plt.plot(epochs, val_losses, label='Val Loss', marker='s')
    plt.title("Loss History")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epochs, rouge_scores, label='Val ROUGE-L', marker='^', color='green')
    plt.title("ROUGE-L Score History")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

def main():
    logging.info("="*50)
    logging.info(f"Starting training Seq2Seq LUONG ATTENTION LSTM on Device: {device}")

    vocab = Vocab(
        path=DATASET_ROOT,
        src_language="english",
        tgt_language="vietnamese"
    )

    train_dataset = phoMTDataset(os.path.join(DATASET_ROOT, "small-train.json"), vocab)
    dev_dataset = phoMTDataset(os.path.join(DATASET_ROOT, "small-dev.json"), vocab)
    test_dataset = phoMTDataset(os.path.join(DATASET_ROOT, "small-test.json"), vocab)

    logging.info(f"Using full datasets: Train size={len(train_dataset)}, Dev size={len(dev_dataset)}, Test size={len(test_dataset)}")

    BATCH_SIZE = 16

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

    # KHỞI TẠO MODEL ATTENTION MỚI
    model = Seq2SeqAttentionLSTM(
        d_model=256,
        n_encoder=3,
        n_decoder=3,
        dropout=0.3,
        vocab=vocab
    ).to(device)

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    logging.info(f"Model Parameters (LSTM with Luong Attention): {total_params:,}")

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss(ignore_index=vocab.pad_idx)

    checkpoint_path = os.path.join(CHECKPOINT_DIR, "best_luong_lstm_mt.pt")
    best_rouge = 0.0
    start_epoch = 0

    if os.path.exists(checkpoint_path):
        try:
            ckpt = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(ckpt['model_state_dict'])
            optimizer.load_state_dict(ckpt['optimizer_state_dict'])
            start_epoch = ckpt['epoch']
            best_rouge = ckpt.get('best_rouge', 0.0)
            logging.info(f"Resumed from epoch {start_epoch}, Best ROUGE: {best_rouge:.4f}")
        except Exception as e:
            logging.warning(f"Error loading checkpoint: {e}. Starting from scratch.")

    train_losses, val_losses, val_rouges = [], [], []
    patience = 0

    for epoch in range(start_epoch + 1, 20):
        logging.info(f"\n--- Starting Epoch {epoch} ---")

        t_loss = train(model, train_loader, epoch, loss_fn, optimizer)
        v_loss, v_rouge = evaluate(model, dev_loader, epoch, loss_fn, vocab)

        train_losses.append(t_loss)
        val_losses.append(v_loss)
        val_rouges.append(v_rouge)

        if v_rouge > best_rouge:
            best_rouge = v_rouge
            patience = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_rouge': best_rouge
            }, checkpoint_path)
            logging.info(f"!!! NEW BEST MODEL (Epoch {epoch}) !!! Saved ROUGE-L: {best_rouge:.4f}")
        else:
            patience += 1
            logging.info(f"No improvement. Patience: {patience}/10")
            if patience >= 10:
                logging.info("Early stopping!")
                break

    logging.info("\n================= Final Test Evaluation =================")
    if os.path.exists(checkpoint_path):
        ckpt = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(ckpt['model_state_dict'])

        test_loss, test_rouge = evaluate(model, test_loader, 0, loss_fn, vocab)
        logging.info(f"Final Test Loss: {test_loss:.4f} | Test ROUGE-L: {test_rouge:.4f}")
    else:
        logging.warning("Cannot evaluate on Test Set: Best model checkpoint not found.")

    visualize_metrics(train_losses, val_losses, val_rouges)

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.error(f"Error: {str(e)}", exc_info=True)
        print(f"\n[FATAL ERROR] Check training.log for details. Error: {str(e)}")

Writing /content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/train_luong_lstm.py


In [ ]:
!python /content/drive/MyDrive/DATA-SCIENCE-SUBJECT/DS201/PRACTICE/DS201-LAB4/train_luong_lstm.py

Starting training Seq2Seq LUONG ATTENTION LSTM on Device: cuda
Using full datasets: Train size=20000, Dev size=2000, Test size=2000
Model Parameters (LSTM with Luong Attention): 6,695,111

--- Starting Epoch 1 ---
Epoch 1 - Training: 100% 1250/1250 [03:57<00:00,  5.27it/s, loss=5.63]
--- Epoch 1 TRAIN finished --- Loss: 5.6264
Epoch 1 - Evaluating:   0% 0/125 [00:00<?, ?it/s]
======== Example Translation (Epoch 1) ========
-> Source (EN):     <unk> <unk> , one of the most powerful storms ever recorded in the Atlantic Ocean , made <unk> as a <unk> 5 storm on Great <unk> Island in the northern <unk> on Sunday morning , September 1 , <unk> .
-> Reference (VN):  Vào chủ nhật ngày <unk> , cơn bão <unk> , một trong những cơn bão mạnh nhất được ghi nhận ở Đại Tây Dương , với sức gió <unk> <unk> đổ bộ vào đảo Great <unk> , miền bắc <unk> .
-> Prediction (VN): <unk> <unk> , <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <u